# 数据科学与工程导论实验报告-12
温兆和 10205501432

首先我们引入在本次实验中需要用到的Python库。

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## 实验背景
在实验练习08中我们使用的数据为两个城市在不同因素影响下某小时共享单车租用的数量。现在我们利用该数据集构建一个回归预测模型，根据数据中的某些属性预测该时段内的单车租用数量，该表所有值均为整型数据，表结构为：
- id 记录编号，无其他意义
- city 城市代号，0为北京，1为上海
- hour 小时，代表时间
- is_workday 是否为工作日，0为否，1为是
- temp_air 大气温度，单位为摄氏度
- temp_body 体感温度，单位为摄氏度
- weather 天气代号，1为晴天，2为多云或阴天，3为雨天或雪天
- wind 风级，数值越大代表风速越大
- y 该小时内共享单车被租用的数量
## 实验一：数据获取
数据集已下载至bike.csv文件中，请使用pandas库读取该文件。
- 提示：原始数据集共10,000条数据。

In [3]:
#读取CSV文件
Data = pd.read_csv("bike.csv")
print(Data.head())

   id  city  hour  is_workday  weather  temp_1  temp_2  wind   y
0   1     0    22           1        2     3.0     0.7     0  15
1   2     0    10           1        1    21.0    24.9     3  48
2   3     0     0           1        1    25.3    27.4     0  21
3   4     0     7           0        1    15.7    16.2     0  11
4   5     1    10           1        1    21.1    25.0     2  39


In [4]:
print(len(Data))

10000


## 实验二：数据预处理 I
id属性对构建回归预测模型没有帮助，请剔除掉该列。

In [5]:
Data1 = Data.drop(labels=['id'], axis = 1)
print(Data1.head())

   city  hour  is_workday  weather  temp_1  temp_2  wind   y
0     0    22           1        2     3.0     0.7     0  15
1     0    10           1        1    21.0    24.9     3  48
2     0     0           1        1    25.3    27.4     0  21
3     0     7           0        1    15.7    16.2     0  11
4     1    10           1        1    21.1    25.0     2  39


## 实验三：数据预处理 II
我们暂不考虑不同城市对单车租用的影响，请筛选出上海市的所有数据，然后剔除city列。
- 提示：目前剩余4,998条数据。

In [6]:
Data2_apostrophe = Data1.loc[Data1['city'] == 1]
Data2 = Data2_apostrophe.drop(labels=['city'], axis = 1)
print(Data2.head())

    hour  is_workday  weather  temp_1  temp_2  wind   y
4     10           1        1    21.1    25.0     2  39
5      0           1        1    20.4    18.2     0  12
9      4           1        3    17.4    18.0     3   2
10     0           1        1    14.9    15.3     2   6
11     8           0        1    25.0    28.1     0  25


In [7]:
print(len(Data2))

4998


## 实验四：数据预处理 III
为简化数据，请将hour列中原来6点-18点统一为1；19点-次日5点统一为0。

In [8]:
Data3 = Data2.copy()
index_list = Data3.index.tolist()
for i in range(len(Data3)):
    if (Data3['hour'][index_list[i]] >=6)&(Data3['hour'][index_list[i]]<=18):
        Data3['hour'][index_list[i]]=1
    else:
        Data3['hour'][index_list[i]]=0
print(Data3.head())

/tmp/ipykernel_2623239/2705789550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data3['hour'][index_list[i]]=1
/tmp/ipykernel_2623239/2705789550.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data3['hour'][index_list[i]]=0


    hour  is_workday  weather  temp_1  temp_2  wind   y
4      1           1        1    21.1    25.0     2  39
5      0           1        1    20.4    18.2     0  12
9      0           1        3    17.4    18.0     3   2
10     0           1        1    14.9    15.3     2   6
11     1           0        1    25.0    28.1     0  25


## 实验五：数据预处理 IV
y列为单车租用数量，是我们的预测目标（标签），请将该列提取出来，并转换为一个numpy列向量，将原先的y列剔除。

In [9]:
#提取y列并转换为numpy
Y_df = Data3['y']
Data_y = Y_df.to_numpy()
print(Data_y)

[39 12  2 ...  1  1 11]


In [10]:
#剔除原先的y列
Data4 = Data3.drop(labels=['y'], axis = 1)
print(Data4.head())

    hour  is_workday  weather  temp_1  temp_2  wind
4      1           1        1    21.1    25.0     2
5      0           1        1    20.4    18.2     0
9      0           1        3    17.4    18.0     3
10     0           1        1    14.9    15.3     2
11     1           0        1    25.0    28.1     0


## 实验六：数据预处理 V
请将DataFrame对象转换为Numpy数组，方便后续操作。

In [11]:
Data5 = Data4.to_numpy()
print(Data5)

[[ 1.   1.   1.  21.1 25.   2. ]
 [ 0.   1.   1.  20.4 18.2  0. ]
 [ 0.   1.   3.  17.4 18.   3. ]
 ...
 [ 0.   1.   3.  13.7 14.1  2. ]
 [ 0.   0.   1.  22.3 22.2  0. ]
 [ 0.   0.   1.   9.6  9.7  0. ]]


## 实验七：数据集划分
请按照训练集与测试集8:2的比例将原始数据集划分。

In [47]:
x_train,x_test,y_train,y_test = train_test_split(Data5,Data_y,test_size=0.2,shuffle=True) 
print('训练集数据量：',len(x_train))
print('测试集数据量：',len(x_test))

训练集数据量： 3998
测试集数据量： 1000


## 实验八：数据预处理 VI
请分别对训练集数据、训练集标签、测试集数据和测试集标签进行归一化。

In [48]:
#训练集数据
min_max_scaler = MinMaxScaler()
x_train_mm = min_max_scaler.fit_transform(x_train)
print(x_train_mm)

[[1.         1.         0.5        0.40848214 0.42375602 0.        ]
 [1.         0.         0.         0.62946429 0.62600321 0.        ]
 [1.         0.         0.         0.61160714 0.57784912 0.        ]
 ...
 [1.         1.         0.         0.53571429 0.52166934 0.28571429]
 [1.         1.         0.         0.74330357 0.71268058 0.        ]
 [1.         1.         0.         0.19642857 0.21348315 0.14285714]]


In [49]:
#训练集标签
y_train=y_train.reshape(-1,1)
y_train_mm = min_max_scaler.fit_transform(y_train)
y_train_mm = y_train_mm.reshape(1,-1)
y_train_mm = y_train_mm[0]
print(y_train_mm)

[0.68062827 0.04712042 0.41884817 ... 0.23560209 0.41884817 0.13089005]


In [50]:
#测试集数据
x_test_mm = min_max_scaler.fit_transform(x_test)
print(x_test_mm)

[[0.         0.         0.5        0.51851852 0.53209459 0.        ]
 [0.         1.         0.         0.37037037 0.36655405 0.6       ]
 [0.         1.         0.         0.78703704 0.77364865 0.2       ]
 ...
 [1.         1.         1.         0.65509259 0.63851351 0.4       ]
 [0.         1.         0.         0.66898148 0.61486486 0.        ]
 [1.         1.         0.         0.59027778 0.58783784 0.        ]]


In [51]:
#测试集标签
y_test=y_test.reshape(-1,1)
y_test_mm = min_max_scaler.fit_transform(y_test)
y_test_mm = y_test_mm.reshape(1,-1)
y_test_mm = y_test_mm[0]
print(y_test_mm)

[0.15384615 0.1025641  0.10769231 0.68205128 0.         0.54358974
 0.01538462 0.02051282 0.45641026 0.40512821 0.34871795 0.53333333
 0.17948718 0.00512821 0.24615385 0.48717949 0.01538462 0.
 0.16923077 0.03589744 0.03076923 0.24615385 0.10769231 0.14871795
 0.02051282 0.39487179 0.40512821 0.30769231 0.58461538 0.01538462
 0.27179487 0.18461538 0.23076923 0.11282051 0.06666667 0.18974359
 0.00512821 0.01025641 0.91282051 0.01025641 0.00512821 0.46153846
 0.37948718 0.17948718 0.01538462 0.05641026 0.00512821 0.28717949
 0.30769231 0.02564103 0.09230769 0.35384615 0.3025641  0.21538462
 0.5025641  0.18461538 0.23589744 0.22051282 0.06666667 0.23589744
 0.75897436 0.0974359  0.06666667 0.35897436 0.17435897 0.32307692
 0.04615385 0.25641026 0.29230769 0.18461538 0.01025641 0.07692308
 0.27692308 0.48205128 0.07692308 0.17948718 0.38974359 0.04615385
 0.12307692 0.05128205 0.01538462 0.06153846 0.06666667 0.03076923
 0.0974359  0.46666667 0.03589744 0.58461538 0.33846154 0.33846154
 0.

## 实验九：模型构建
请先构建一个线性回归模型（多元一次函数），然后利用训练集训练模型。

In [52]:
model = Pipeline([('Poly',PolynomialFeatures(degree=1)),('Linear',LinearRegression())])
model.fit(x_train_mm,y_train_mm)
print(model.named_steps['Linear'].coef_)
print(model.named_steps['Linear'].intercept_)

[ 0.          0.17240615 -0.00184708 -0.08401013  0.12877232  0.31260213
  0.03792335]
-0.07401273011315398


## 实验十：模型测试
利用测试集对训练好的模型进行评估。
- 提示：使用predict(data_array)方法输入测试集，该函数返回值为模型预测值。

In [53]:
y_predict = model.predict(x_test_mm)
print(y_predict)

[ 0.11708694  0.10917323  0.27491765  0.32291448  0.00805777  0.29521663
  0.13575422  0.0593953   0.2288419   0.23654078  0.26918187  0.31174015
  0.25165411  0.12612904  0.26372727  0.29517072  0.22464681  0.12058488
  0.09022605  0.18863874  0.13568514  0.26838091  0.29395694  0.26537953
  0.19276938  0.28434149  0.36409663  0.51823458  0.54343213  0.03549061
  0.29153359  0.14394643  0.403816    0.1054407   0.05058067  0.18368197
  0.14292688  0.03111566  0.43313473  0.15127832  0.15258624  0.23890495
  0.41850765  0.40085979  0.20321417  0.2686323   0.20155871  0.3277698
  0.413939   -0.02035984  0.23726558  0.25176048  0.25633743  0.24502441
  0.28758605  0.25232609  0.23153249  0.1235913   0.15694372  0.53835606
  0.50670764  0.22825188  0.24075324  0.26042276  0.12520609  0.43829125
  0.20987022  0.05875736  0.12238665  0.43956497  0.26973508  0.26237622
  0.24374114  0.49781506  0.15337026  0.49009504  0.28884024 -0.08202274
  0.03088215  0.04116004  0.0959496   0.02380443  0.

## 实验十一：模型评估
请使用均方根误差（$RMSE$）作为评估指标，并输出$RMSE$值。
- 提示：均方根误差(Root of Mean Squared Error)，可以直接通过对$MSE$求平方根获得。公式为
$$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (\hat{y_i}-y_i)^2}$$

In [54]:
rmse = np.sqrt(np.mean((y_predict-y_test_mm)**2))
print("The RMSE value is", round(rmse,4))

The RMSE value is 0.166
